In [1]:
%pip install langchain_community langchain langchain_cohere langchain_groq

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [4]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


In [17]:
import os 

GROQ_API_KEY='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'
os.environ["GROQ_API_KEY"]='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'

In [22]:
from langchain_groq import ChatGroq
llm= ChatGroq(
            model='llama3-8b-8192',
            temperature=0.6,
            api_key=os.getenv("GROQ_API_KEY")
        )

In [20]:
from langchain.llms import Ollama
llm =Ollama(model='llama3.2:3b') 

In [9]:
NEO4J_URI='neo4j+s://ac661595.databases.neo4j.io'
NEO4J_USERNAME='neo4j'
NEO4J_PASSWORD='CZpLyBr6A_cxF15QEKbQX6mUxblAPxLEXWva6r3_66c'


In [10]:
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [11]:

from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph

In [12]:
graph = Neo4jGraph()


In [13]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = r"C:\Users\user\Desktop\Alchemist AI Assignment\data"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder.")


Loaded 1 documents from the folder.


In [14]:
embeddings_model_name='embed-english-v2.0'
COHERE_API_KEY='30lAzH7VO1y9M00rSD5ns4gIIxicqxiMnCn1IycB'
import os
os.environ['COHERE_API_KEY']=COHERE_API_KEY 
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
        model=embeddings_model_name,
        cohere_api_key=os.getenv('COHERE_API_KEY')
    )

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\user\AppData\Local\sagemaker\sagemaker\config.yaml


In [15]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")


Split the documents into 115 chunks.


In [23]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [24]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [25]:
graph_documents


[GraphDocument(nodes=[Node(id='Mughal Empire', type='Dynasty', properties={}), Node(id='Babur', type='Person', properties={}), Node(id='Akbar The Great', type='Person', properties={}), Node(id='Shah Jahan', type='Person', properties={}), Node(id='Aurangzeb', type='Person', properties={}), Node(id='Timur', type='Person', properties={}), Node(id='Genghis Khan', type='Person', properties={}), Node(id='Mumtaz Mahal', type='Person', properties={}), Node(id='Bahadur Shah Ii', type='Person', properties={})], relationships=[Relationship(source=Node(id='Babur', type='Person', properties={}), target=Node(id='Mughal Empire', type='Dynasty', properties={}), type='ESTABLISHED', properties={}), Relationship(source=Node(id='Akbar The Great', type='Person', properties={}), target=Node(id='Mughal Empire', type='Dynasty', properties={}), type='RULED', properties={}), Relationship(source=Node(id='Shah Jahan', type='Person', properties={}), target=Node(id='Mughal Empire', type='Dynasty', properties={}), t

In [26]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [27]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [28]:
from langchain_community.vectorstores import Neo4jVector
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [29]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [30]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


c:\Users\user\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [33]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [34]:
entities=entity_chain.invoke({"question": "Who was Babur?"}).names


In [35]:
entities

['Babur']

In [36]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [37]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [41]:
print(structured_retriever("Who built the Taj Mahal?"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


Mughal Empire - COMMISSIONED -> Taj Mahal


In [42]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [43]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [44]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [45]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [46]:
from typing import Tuple, List, Optional
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [47]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [48]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | llm
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [49]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [50]:
prompt = ChatPromptTemplate.from_template(template)

In [51]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [53]:
chain.invoke({"question": "Who was Akbar ?"})

Search query: Who was Akbar ?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

'Akbar was the ruler of the Mughal Empire.'

In [54]:
chain.invoke(
    {
        "question": "Who was Akbar's father?",
        
    }
)

Search query: Who was Akbar's father?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

'Babur.'

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from typing import Tuple, List

# Step 1: Set up the graph transformer
def setup_graph_transformer(llm, documents, graph):
    """Set up the LLM-based graph transformer and add documents to the graph."""
    llm_transformer = LLMGraphTransformer(llm=llm)
    graph_documents = llm_transformer.convert_to_graph_documents(documents)
    graph.add_graph_documents(
        graph_documents,
        baseEntityLabel=True,
        include_source=True
    )

# Step 2: Initialize Neo4j vector store
def initialize_vector_store(embeddings):
    """Initialize a Neo4j vector store with specified embeddings."""
    return Neo4jVector.from_existing_graph(
        embeddings,
        search_type="hybrid",
        node_label="Document",
        text_node_properties=["text"],
        embedding_node_property="embedding"
    )

# Step 3: Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""
    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that appear in the text.",
    )

def create_entity_extraction_prompt(llm):
    """Create a prompt for extracting entities."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are extracting organization and person entities from the text.",
            ),
            (
                "human",
                "Use the given format to extract information from the following input: {question}",
            ),
        ]
    )
    return prompt | llm.with_structured_output(Entities)

# Step 4: Generate full-text query for graph search
def generate_full_text_query(input: str) -> str:
    """Generate a full-text query for Neo4j."""
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Step 5: Structured retrieval from Neo4j
def structured_retriever(question: str, graph, entity_chain) -> str:
    """Retrieve structured data from Neo4j based on entities in the question."""
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

# Step 6: Unified retriever combining structured and unstructured data
def retriever(question: str, graph, vector_index, entity_chain):
    """Retrieve data from structured and unstructured sources."""
    print(f"Search query: {question}")
    structured_data = structured_retriever(question, graph, entity_chain)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
    {structured_data}
    Unstructured data:
    {"#Document ". join(unstructured_data)}
    """
    return final_data

# Step 7: Create a query condensing and answering pipeline
def format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    """Format chat history into a list of messages."""
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

def create_condense_question_prompt() -> PromptTemplate:
    """Create a prompt for condensing follow-up questions."""
    _template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
    return PromptTemplate.from_template(_template)

def create_search_query_pipeline(llm, condense_prompt: PromptTemplate) -> RunnableBranch:
    """Create a pipeline to process search queries."""
    return RunnableBranch(
        (
            RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
                run_name="HasChatHistoryCheck"
            ),
            RunnablePassthrough.assign(
                chat_history=lambda x: format_chat_history(x["chat_history"])
            )
            | condense_prompt
            | llm
            | StrOutputParser(),
        ),
        RunnableLambda(lambda x: x["question"]),
    )

def create_final_prompt_template() -> ChatPromptTemplate:
    """Create a final prompt template for answering questions."""
    template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
    return ChatPromptTemplate.from_template(template)

def create_query_pipeline(llm) -> RunnableParallel:
    """Create a pipeline for condensing questions and answering."""
    condense_prompt = create_condense_question_prompt()
    search_query_pipeline = create_search_query_pipeline(llm, condense_prompt)
    final_prompt = create_final_prompt_template()

    return (
        RunnableParallel(
            {
                "context": search_query_pipeline | retriever,
                "question": RunnablePassthrough(),
            }
        )
        | final_prompt
        | llm
        | StrOutputParser()
    )



In [56]:
question=input()

In [57]:
def main(llm, documents, graph, embeddings, question):
    """Main function to run the retrieval pipeline."""
    setup_graph_transformer(llm, documents, graph)
    vector_index = initialize_vector_store(embeddings)
    entity_chain = create_entity_extraction_prompt(llm)
    pipeline = create_query_pipeline(llm)
    answer = pipeline.invoke({"question": question})
    return answer

In [60]:
answer=main(llm,documents,graph,embeddings,question)

[#CDBE]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.126.161.242', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687))): OSError('No data')
Unable to retrieve routing information
Transaction failed and will be retried in 0.8825773787257747s (Unable to retrieve routing information)


TypeError: retriever() missing 3 required positional arguments: 'graph', 'vector_index', and 'entity_chain'